<a href="https://colab.research.google.com/github/amanjain1397/zip-the-code-1.0/blob/master/v_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -i https://test.pypi.org/simple/ supportlib

Looking in indexes: https://test.pypi.org/simple/


In [0]:
import supportlib.gettingdata as getdata
getdata.kaggle()


Saving kaggle.json to kaggle.json


In [0]:
!kaggle datasets download -d thanakomsn/glove6b300dtxt

 98% 378M/386M [00:08<00:00, 25.8MB/s]
100% 386M/386M [00:09<00:00, 44.6MB/s]


In [0]:
getdata.zipextract('/content/glove6b300dtxt.zip')

In [0]:
!git clone https://github.com/ipmob/NITD-machine-Learning-challange

Cloning into 'NITD-machine-Learning-challange'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import keras

import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def preprocess_text(text):
    
    #Lowering case the text
    text = text.lower()
    
    #Removing numbers
    text = re.sub(r'\d+', '', text)
    
    #Removing punctuations
    import string
    from nltk.tokenize import word_tokenize
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Removing stop words
    
    tokens = word_tokenize(text)
    text = ' '.join([ i for i in tokens if not i in stopwords])
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    return text

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
data = pd.read_csv('/content/NITD-machine-Learning-challange/data/train_file.csv')
test = pd.read_csv('/content/NITD-machine-Learning-challange/data/test_file.csv')
sample = pd.read_csv('/content/NITD-machine-Learning-challange/data/results_file.csv')

data.loc[data.Subjects.isnull(), 'Subjects'] = ''
test.loc[test.Subjects.isnull(), 'Subjects'] = ''

data['combined'] = data.Subjects.str.cat(' ' + data.Title)
test['combined'] = test.Subjects.str.cat(' ' + test.Title)

dataMatType = data.MaterialType
data.drop(labels = ['MaterialType'], axis = 1, inplace = True)

data.combined = data.combined.apply(preprocess_text)
test.combined = test.combined.apply(preprocess_text)
'''
data.Subjects = data.Subjects.apply(preprocess_text)
data.Title = data.Title.apply(preprocess_text)
test.Subjects = test.Subjects.apply(preprocess_text)
test.Title = test.Title.apply(preprocess_text)
'''
flag = -1
labelDict = {}

for key in dataMatType.value_counts().keys():
  flag += 1
  labelDict[key] = flag
  
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 25                             

max_words = 1000                                    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.combined)

dataSequences = tokenizer.texts_to_sequences(data.combined)
testSequences = tokenizer.texts_to_sequences(test.combined)

data_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(data_word_index))

Found 36088 unique tokens.


In [0]:
from keras import preprocessing
x_train = preprocessing.sequence.pad_sequences(dataSequences, maxlen = 25)
x_test = preprocessing.sequence.pad_sequences(testSequences, maxlen = 25)

train_label = dataMatType.map(labelDict)

from keras.utils import to_categorical
labelDataBinary = to_categorical(train_label)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(dataMatType), dataMatType)

In [0]:
import os


embeddings_index = {}
f = open('/content/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 500

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in data_word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model = Sequential()
model.add(Embedding(1000, 1000 ,input_length=25))



model.add(Conv1D(256, 5, activation='elu'))
model.add(Dropout(0.6))
model.add(Conv1D(512, 5, activation='elu'))
model.add(Dropout(0.6))

model.add(Conv1D(1024, 5, activation='elu'))
model.add(Dropout(0.6))

model.add(Flatten())

model.add(Dense(400, activation='elu'))
model.add(Dense(100, activation='elu'))
model.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.0005)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=20,
                    validation_split=0.2,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model.save('m1.h5')

Train on 25322 samples, validate on 6331 samples
Epoch 1/20
25322/25322 [==============================] - 38s 1ms/step - loss: 0.6301 - acc: 0.8253 - val_loss: 0.4674 - val_acc: 0.8594
Epoch 2/20
25322/25322 [==============================] - 29s 1ms/step - loss: 0.4663 - acc: 0.8649 - val_loss: 0.4822 - val_acc: 0.8724
Epoch 3/20
25322/25322 [==============================] - 29s 1ms/step - loss: 0.4482 - acc: 0.8704 - val_loss: 0.4394 - val_acc: 0.8741
Epoch 4/20
25322/25322 [==============================] - 29s 1ms/step - loss: 0.3959 - acc: 0.8774 - val_loss: 0.4479 - val_acc: 0.8752
Epoch 5/20
25322/25322 [==============================] - 29s 1ms/step - loss: 0.3720 - acc: 0.8790 - val_loss: 0.4670 - val_acc: 0.8740
Epoch 6/20
25322/25322 [==============================] - 29s 1ms/step - loss: 0.3000 - acc: 0.8965 - val_loss: 0.4478 - val_acc: 0.8807
Epoch 7/20
25322/25322 [==============================] - 29s 1ms/step - loss: 0.2771 - acc: 0.9021 - val_loss: 0.4650 - val_acc:

KeyboardInterrupt: ignored

In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model = Sequential()
model.add(Embedding(1000, 1000 ,input_length=25))


model.add(CuDNNGRU(600, return_sequences = True))
model.add(Dropout(0.6))

model.add(CuDNNGRU(600, return_sequences = True))
model.add(Dropout(0.6))

model.add(CuDNNGRU(800))
model.add(Dropout(0.7))

model.add(Dense(200, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=20,
                    validation_split=0.2,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model.save('m1.h5')

Train on 25322 samples, validate on 6331 samples
Epoch 1/20
 3392/25322 [===>..........................] - ETA: 1:39 - loss: 0.7891 - acc: 0.7969

In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model = Sequential()
model.add(Embedding(1000, 1000 ,input_length=25))


model.add(CuDNNGRU(600, return_sequences = True))
model.add(Dropout(0.6))

model.add(CuDNNGRU(600))
model.add(Dropout(0.6))

model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=20,
                    validation_split=0.2,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model.save('m1.h5')

Train on 25322 samples, validate on 6331 samples
Epoch 1/20
25322/25322 [==============================] - 45s 2ms/step - loss: 0.5030 - acc: 0.8604 - val_loss: 0.4077 - val_acc: 0.8822
Epoch 2/20
25322/25322 [==============================] - 37s 1ms/step - loss: 0.4008 - acc: 0.8803 - val_loss: 0.4030 - val_acc: 0.8836
Epoch 3/20
25322/25322 [==============================] - 37s 1ms/step - loss: 0.3687 - acc: 0.8857 - val_loss: 0.3952 - val_acc: 0.8841
Epoch 4/20
25322/25322 [==============================] - 37s 1ms/step - loss: 0.3444 - acc: 0.8931 - val_loss: 0.3878 - val_acc: 0.8874
Epoch 5/20
25322/25322 [==============================] - 37s 1ms/step - loss: 0.3231 - acc: 0.8948 - val_loss: 0.4050 - val_acc: 0.8896
Epoch 6/20
25322/25322 [==============================] - 37s 1ms/step - loss: 0.3067 - acc: 0.9008 - val_loss: 0.4206 - val_acc: 0.8864
Epoch 7/20
25322/25322 [==============================] - 38s 1ms/step - loss: 0.2568 - acc: 0.9114 - val_loss: 0.4383 - val_acc:

KeyboardInterrupt: ignored

In [0]:
#model.save('./models/m1.h5')
model.save('m1.h5')

**Using Embedding Layer**

In [0]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(Embedding(1000, 300, input_length=10, weights=[embedding_matrix], trainable=False))

'''model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''

model.add(LSTM(1000, recurrent_dropout = 0.3))
model.add(Dropout(0.35))
model.add(Dense(8, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, labelDataBinary,
                    epochs=8,
                    batch_size=128,
                    validation_split=0.2,
                    class_weight = class_weights)

Train on 25322 samples, validate on 6331 samples
Epoch 1/8
25322/25322 [==============================] - 45s 2ms/step - loss: 0.5646 - acc: 0.8440 - val_loss: 0.4672 - val_acc: 0.8574
Epoch 2/8
25322/25322 [==============================] - 42s 2ms/step - loss: 0.4551 - acc: 0.8647 - val_loss: 0.4274 - val_acc: 0.8686
Epoch 3/8
25322/25322 [==============================] - 41s 2ms/step - loss: 0.4218 - acc: 0.8710 - val_loss: 0.4192 - val_acc: 0.8711
Epoch 4/8
25322/25322 [==============================] - 40s 2ms/step - loss: 0.3957 - acc: 0.8754 - val_loss: 0.4168 - val_acc: 0.8724
Epoch 5/8
25322/25322 [==============================] - 45s 2ms/step - loss: 0.3713 - acc: 0.8809 - val_loss: 0.4109 - val_acc: 0.8768
Epoch 6/8
25322/25322 [==============================] - 45s 2ms/step - loss: 0.3469 - acc: 0.8870 - val_loss: 0.4196 - val_acc: 0.8751
Epoch 7/8
25322/25322 [==============================] - 49s 2ms/step - loss: 0.3189 - acc: 0.8929 - val_loss: 0.4198 - val_acc: 0.8757

In [0]:
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
  
predictions = model.predict(x_test)
predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
predictions = predictions.map(reverseLabelDict)

submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']

In [0]:
submission.to_csv('sub-dl.csv', index = False, header = True)

In [0]:
def predict(model, x_test):
  predictions = model.predict(x_test)
  predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
  predictions = predictions.map(reverseLabelDict)
  return predictions

In [0]:
#num_model = 5
predic1 = pd.Series(predict(model1, x_test))
#predic2 = pd.Series(predict(model2, x_test))
#predic3 = pd.Series(predict(model3, x_test))
#predic4 = pd.Series(predict(model4, x_test))
#predic5 = pd.Series(predict(model5, x_test))

In [0]:
#total = pd.DataFrame(pd.concat([predic1, predic2, predic3, predic4, predic5], axis = 1))
total = pd.DataFrame(pd.concat([predic1], axis = 1))
submissions = total.mode(axis = 1)[0]
submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('sub-mew.csv', index = False, header = True)

In [0]:
submissions = total.mode(axis = 1)[0]